In [1]:
import sympy as sp
from sympy.utilities.lambdify import lambdify, lambdastr

import numpy as np
np.seterr(all='raise')

from scipy import optimize as opt
from sympy import Matrix, MatrixSymbol, refine, Identity, Q
from tkinter import *
import time

sp.init_printing()
from nbsupport import md

# AGI の実装

AGI (Active Graph Interface) とは、対話的な高次元グラフ配置方式である。
大まかな流れとしては、

1. グラフの距離行列と辺のリストを入力とする。
2. 多次元尺度構成法により、高次元の座標系と２次元への射影ベクトルを生成する。
3. 2で生成したものを用いて、２次元の座標系を生成する。
4. ２次元の座標系での操作により、グラフを適切に再配置する。

のようになる。

（gifなどを貼り付けてイメージを与える）

では、それらについてグラフの配置方法を簡単な例に沿って見ていく。

## 1. グラフの距離行列と辺のリスト

グラフ距離と辺のリストは入力として与える。グラフ距離はMDS 法において、辺のリストは最終的な描画において使用する。

In [2]:
EdgeList = np.genfromtxt('csv/edgeList.csv', delimiter=",").astype(np.int64)
edge_num = len(EdgeList)

## 2. 多次元尺度構成法（MDS法）

MDS法とは多次元点間の距離が与えられた時に、その距離を再現するような座標系を逆算する手法である。
ここで、与えられたグラフの距離行列を D とする。(D は (n,n) 型行列)

この距離行列の i,j 成分を d(i,j) とし、以下のような i,j 成分を持つ行列 A を定義する。

a(i,j) = ...

この行列 A の固有値(l_1,l_2,...l_n)を大きい順に並べる。また、この並べた固有値で負の値を取るものは 0 に置き換える。このようなリストを eList とする。このリストを対角に並べた行列を L とする。A の固有ベクトルを縦ベクトルで並べた行列を V とする。
さらに、L の全ての要素の平方を取った行列を L' とした時、求める高次元配置は以下のような X である。

X = V * L'

正である A の固有値の数が p 個である時、X は p 次元の座標系である。
これらの計算は R 上で行い、データは csv ファイルに書き出しておく。

In [3]:
mds_file = 'csv/mdSpace.csv'
MDS = np.genfromtxt(mds_file, delimiter=",")
node_num, high_dim = MDS.shape
md('記号 MDS は $ ',mds_file,' $ のファイルに保存された高次元空間の座標系が代入されている。',
   'この MDS の座標の次元は $ ',high_dim,' $ であり、座標の数は $ ',node_num,' $ である。')

記号 MDS は $ csv/mdSpace.csv $ のファイルに保存された高次元空間の座標系が代入されている。この MDS の座標の次元は $ 12 $ であり、座標の数は $ 15 $ である。

## 3. ２次元の座標系への変換

高次元配置 X に射影ベクトルを掛けることで２次元に投射する。射影ベクトルは以下のように定義する。

射影ベクトル： E1, E2
s.t. 行列 A の固有値の列 eList = (l1, l2, l3, ..., lp, 0, ..., 0)とした時、
E1 = (l1, 0, l3, 0, l5, ...)
E2 = (0, l2, 0, l4, 0, ...)
のように、偶奇で振り分ける。

In [4]:
# generate projection vectors
eList = np.sqrt(np.genfromtxt('csv/eigVals.csv', delimiter=",")[0:high_dim])
base = np.zeros(high_dim * 2).reshape(2, high_dim)
e0_column = np.zeros(high_dim).reshape(1,high_dim)
for i in range(high_dim): base[i % 2][i] = 1
Es = np.r_[base*eList, e0_column]

md('射影ベクトルは固有値のリストを元にして生成する。上記の固有値の列 $',eList,'$ について、',
  '奇数番目と偶数番目を振り分けることで２つの射影ベクトル $',Es[0],'$ と $',Es[1],'$ を生成する。')

Es = Es.T

射影ベクトルは固有値のリストを元にして生成する。上記の固有値の列 $[  4.12843841e+00   4.12843841e+00   7.07106781e-01   7.07106781e-01
   7.07106781e-01   7.07106781e-01   7.07106781e-01   7.07106781e-01
   7.07106781e-01   7.07106781e-01   7.07106781e-01   2.10734243e-08]$ について、奇数番目と偶数番目を振り分けることで２つの射影ベクトル $[ 4.12843841  0.          0.70710678  0.          0.70710678  0.
  0.70710678  0.          0.70710678  0.          0.70710678  0.        ]$ と $[  0.00000000e+00   4.12843841e+00   0.00000000e+00   7.07106781e-01
   0.00000000e+00   7.07106781e-01   0.00000000e+00   7.07106781e-01
   0.00000000e+00   7.07106781e-01   0.00000000e+00   2.10734243e-08]$ を生成する。

射影ベクトルを生成したら、それと高次元座標系の積を取ることで平面の座標系を生成する。

In [5]:
Position = np.array([])

def update_points():
    global Position
    Position = MDS.dot(Es[:,0:2])
    
update_points()

print(Position)

[[ 0.01714159 -2.48296658]
 [ 0.42390324 -6.82746494]
 [-0.22380977 -6.56967166]
 [ 0.28604207 -6.5813416 ]
 [-0.29707523 -7.40699408]
 [ 2.14174134  1.25632834]
 [ 6.42896614  3.3669566 ]
 [ 6.24725276  3.24439124]
 [ 5.59683338  3.45586181]
 [ 5.34893225  3.78925753]
 [-2.15888293  1.22663824]
 [-5.94662269  3.0583117 ]
 [-6.07612649  3.33630223]
 [-5.75080351  3.88309129]
 [-6.03749215  3.25129988]]


また、AGI の最大の特徴は動的なグラフ配置の更新である。
その更新によって、我々は高次元グラフを様々な角度から観察することが出来る。

## 4. 配置の更新

制約解消系として、以下の４つ

1. 新しい射影ベクトルは正規直行
2. 回転の軸同士は正規直行
3. 回転の軸について、射影ベクトルは常に
4. 新しい射影ベクトルによって定められた座標が実際に更新された座標である。

の制約の条件に基づいて式を立てる。

（コードと対応して書くにはどうすれば？） 

In [6]:
# 代数の用意
a1,b1,c1,a2,b2,c2,t,s = sp.symbols('a1 b1 c1 a2 b2 c2 t s')   # variables
x2_s,y2_s = sp.symbols('x2_s y2_s')  # values
P_i = sp.MatrixSymbol('P_i', high_dim, 1)
E1 = sp.MatrixSymbol('E1', high_dim, 1)
E2 = sp.MatrixSymbol('E2', high_dim, 1)
var = (x2_s,y2_s,P_i,E1,E2,a1,b1,c1,a2,b2,c2,t,s)

# 求める E1', E2' と、回転軸の R はベクトル値
_E1 = sp.Matrix(a1*E1 + b1*E2 + c1*P_i)
_E2 = sp.Matrix(a2*E1 + b2*E2 + c2*P_i)
R = sp.Matrix(s*E1 + t*E2)

# 制約の式
f = Matrix([
		_E1.T * _E1 - Matrix([1]),
		_E2.T * _E2 - Matrix([1]),
		_E1.T * _E2,
		R.T * R - Matrix([1]),
		_E1.T * R - sp.Matrix(E1).T * R,
		_E2.T * R - sp.Matrix(E2).T * R,
		sp.Matrix(P_i).T * (_E1) - Matrix([x2_s]),
		sp.Matrix(P_i).T * (_E2) - Matrix([y2_s])
		])

この式を満たす E1', E2' を得るには式を解かねばならないため、近似会を得ることを考える。ここで、右辺が 0 になるように各制約式を式変形し、各制約式の左辺の二乗和を最小化すれば近似できる。

# AGI3D に拡張

上記の AGI と同様の議論によって、AGI は３次元に拡張できる。

- 高次元座標系の生成までは一緒
- 射影ベクトルを３本にする
- 画面更新の際の制約式を３次元版に拡張

制約充足問題は以下のように拡張できる。

In [ ]:
# 変数、定数の定義
a1, b1, c1, d1, a2, b2, c2, d2, a3, b3, c3, d3, t1, s1, u1, t2, s2, u2 = sp.symbols(
	'a1 b1 c1 d1 a2 b2 c2 d2 a3 b3 c3 d3 t1 s1 u1 t2 s2 u2')  # variables
x2_s, y2_s, z2_s = sp.symbols('x2_s y2_s z2_s')  # values
P_i = sp.MatrixSymbol('P_i', high_dim, 1)
E1 = sp.MatrixSymbol('E1', high_dim, 1)
E2 = sp.MatrixSymbol('E2', high_dim, 1)
E3 = sp.MatrixSymbol('E3', high_dim, 1)
_var = (x2_s, y2_s, z2_s, P_i, E1, E2, E3, a1, b1, c1, d1, a2, b2, c2, d2, a3, b3, c3, d3, t1, s1, u1, t2, s2, u2)

E0 = sp.Matrix(P_i - x2_s * E1 - y2_s * E2 - z2_s * E3)
E0 = sp.simplify(E0 / sp.Matrix.norm(E0))

_E1 = sp.Matrix(a1 * E1 + b1 * E2 + c1 * E3 + d1 * E0)
_E2 = sp.Matrix(a2 * E1 + b2 * E2 + c2 * E3 + d2 * E0)
_E3 = sp.Matrix(a3 * E1 + b3 * E2 + c3 * E3 + d3 * E0)
R1 = sp.Matrix(t1 * E1 + s1 * E2 + u1 * E3)
R2 = sp.Matrix(t2 * E1 + s2 * E2 + u2 * E3)

# 制約充足問題
_f = Matrix([
	_E1.dot(_E1) - 1,
	_E2.dot(_E2) - 1,
	_E3.dot(_E3) - 1,
	_E1.dot(_E2),
	_E2.dot(_E3),
	_E3.dot(_E1),
	R1.dot(R1) - 1,
	R2.dot(R2) - 1,
	R1.dot(R2),
	_E1.dot(R1) - sp.Matrix(E1).dot(R1),
	_E2.dot(R1) - sp.Matrix(E2).dot(R1),
	_E3.dot(R1) - sp.Matrix(E3).dot(R1),
	_E1.dot(R2) - sp.Matrix(E1).dot(R2),
	_E2.dot(R2) - sp.Matrix(E2).dot(R2),
	_E3.dot(R2) - sp.Matrix(E3).dot(R2),
	sp.Matrix(P_i).dot(_E1) - x2_s,
	sp.Matrix(P_i).dot(_E2) - y2_s,
	sp.Matrix(P_i).dot(_E3) - z2_s
])

# 上式の二乗和をとり、numpy の式に直す。
_func = sp.Matrix.norm(_f)
_lam_f = lambdify(_var, _func, 'numpy')

# 一般次元の AGI

記述の方法に注意すれば同様に拡張可能。

# コード(必要？)

In [2]:
#  initialize（画像処理関係）
_width = _height = 700  # window's width and height
width = height = 500  # canvas's width and height
eV = np.array([[0,1]])  # Vertical Vector
eH = np.array([[1,0]])  # Horizontal Vector

def scale(pnt,bool):  # データの座標を射影する平面の画面サイズに合わせる
	if(bool): return width * (pnt + boundingH / 2) / boundingH + (_width - width) / 2
	else: return (height - 100) * (boundingV / 2 - pnt) / boundingV + (_height - height) / 2
def unscale(pnt,bool):  # 射影された平面上の座標を元のスケールに戻す
	if (bool): return boundingH * ((pnt - (_width - width) / 2) - width / 2) / width
	else: return boundingV * ((pnt - (_height - height) / 2) - (height - 100) / 2) / (100 - height)

In [4]:
# initialize(データ処理関係)
# load adjacency and multi-dimensional space
EdgeList = np.genfromtxt('csv/edgeList.csv', delimiter=",").astype(np.int64)
edge_num = len(EdgeList)
MDS = np.genfromtxt('csv/mdSpace.csv', delimiter=",")
node_num, high_dim = MDS.shape

low_dim = 2  # この次元のAGIを実行する

In [5]:
# generate projection vectors
def genE():
    L = np.sqrt(np.genfromtxt('csv/eigVals.csv', delimiter=",")[0:high_dim])
    base = np.zeros(high_dim * low_dim).reshape(low_dim, high_dim)
    e0_column = np.zeros(high_dim).reshape(1,high_dim)
    for i in range(high_dim): base[i % low_dim][i] = 1
    E = np.r_[base*L, e0_column]
    return E.T  # 縦ベクトル

Es = genE()  # 射影ベクトルを縦ベクトルで格納(low_dim行が射影ベクトルで、もう１行がベクトル)

Pos_origin = np.zeros(node_num*low_dim).reshape(node_num,low_dim)  # 計算するデータの実際の座標
Pos_scaled = np.zeros(node_num*low_dim).reshape(low_dim,node_num)  # 画面サイズに合わせたデータの座標
boundingV = 0  # Vertical boundary
boundingH = 0  # Horizontal boundary

def scale(pnt,bool):
	if(bool): return width * (pnt + boundingH / 2) / boundingH + (_width - width) / 2
	else: return (height - 100) * (boundingV / 2 - pnt) / boundingV + (_height - height) / 2

def unscale(pnt,bool):
	if (bool): return boundingH * ((pnt - (_width - width) / 2) - width / 2) / width
	else: return boundingV * ((pnt - (_height - height) / 2) - (height - 100) / 2) / (100 - height)

def update_points():
    global Pos_origin, boundingH, boundingV
    Pos_origin = MDS.dot(Es[:,0:low_dim])
    boundingH = max([np.amax(Pos_origin[:,0]), abs(np.amin(Pos_origin[:,0]))]) * 2
    boundingV = max([np.amax(Pos_origin[:,1]), abs(np.amin(Pos_origin[:,1]))]) * 2
    for i in range(node_num):
        Pos_scaled[0,i] = scale(Pos_origin[i,0], True);Pos_scaled[1,i] = scale(Pos_origin[i,1], False)

update_points()

init: ready


In [6]:
# sympy
q =         MatrixSymbol('q', 1, low_dim)  # updated position
p_i =       MatrixSymbol('p_i', 1, high_dim)  # selected point (high_dim) (P[thisID])
E =         MatrixSymbol('E', high_dim,high_dim) # = (e1 e2 e0 0 ...) : 縦ベクトルの列
A_inputEs = MatrixSymbol('A_inputEs', low_dim + 1, low_dim)  # ei' の係数行列
A_inputRs = MatrixSymbol('A_inputRs', low_dim, low_dim - 1)  # Ri の係数行列
var = (q, p_i, E, A_inputEs, A_inputRs)  # 変数のリスト
A = Matrix(np.zeros(high_dim*high_dim).reshape(high_dim,high_dim))
A[0:low_dim + 1, 0:low_dim] = Matrix(A_inputEs)
A[0:low_dim, low_dim:low_dim + 1] = Matrix(A_inputRs)

_E = E * A  # = (e1' e2' R1 0 ...) : 更新後の射影ベクトルは縦ベクトルで格納

# values    ... (q:動かされた先の座標, p_i:動かされている点の高次元座標, E:射影ベクトル)
# variables ... A : 新しい射影ベクトルを構成する係数行列


# 制約解消における前計算
constraints1 = (refine(_E.T * _E, Q.orthogonal(E))).doit()
constraints2 = (refine(E.T * _E, Q.orthogonal(E))).doit()
constraints3 = p_i * _E

# constraints1:
#                                              [ ||e1'||  (e1',e2') (e1',R) 0 ...
# (e1' e2' R 0 ... 0).T * (e1' e2' R 0 ...0) =  (e2',e1') ||e2'||   (e2',R) 0 ...
#                                               ( R ,e1') (e0,e2')   ||R||  0 ...                   ]
#                                                0          0         0     0 ... 
#                                               ...                             ]

# constraints2:
#                                             [(e1,e1') (e1,e2') (e1, R ) 0 ...
# (e1 e2 e0 0 ... 0).T * (e1' e2' R 0 ...0) =  (e2,e1') (e2,e2') (e2, R ) 0 ...
#                                              (e0,e1') (e0,e2') (e0, R ) 0 ...                   ]
#                                                0         0        0     0 ... 
#                                               ...                             ]

# constraints3:
# p_i * (e1' e2' R 0 ...0) =  (x' y' p_i*R 0 ... 0)


# ei' * ej' = δij (クロネッカーのデルタ)
bases_e = Matrix(constraints1[0:low_dim, 0:low_dim] - Matrix(Identity(low_dim)))
_bases_e = bases_e[0,0]**2 + bases_e[0,1]**2 + bases_e[1,1]**2

# Ri * Rj = δij (クロネッカーのデルタ)
bases_r = Matrix(constraints1[low_dim:2 * low_dim - 1, low_dim:2 * low_dim - 1] - Matrix(Identity(low_dim - 1)))
_bases_r = bases_r[0,0]**2

# _Ei.dot(Rj) - sp.Matrix(Ei).dot(Rj),
eMulR =  Matrix(constraints1[0:low_dim, low_dim:2 * low_dim - 1] - constraints2[0:low_dim, low_dim:2 * low_dim - 1])
_eMulR = eMulR[0,0]**2 + eMulR[1,0]**2

# sp.Matrix(P_i).dot(_Ej) - wj
pew = Matrix(constraints3[0, 0:low_dim] - q)
_pew = pew[0,0]**2 + pew[0,1]**2

func = _bases_e + _bases_r + _eMulR + _pew  # 最小二乗法
lam_f = lambdify(var, func, 'numpy')

In [7]:
def lam(q_, P_i, Esub):
    """
    q_    :(1,low_dim) ドラッグされた行先の点の座標
    Esub  :(high_dim,low_dim+1) 画面が更新される前の基底とe0の行列
    P_i   :(1,high_dim) ドラッグされた点に対応する高次元座標
    """
    # a[:,0:3]
    E_ = np.zeros(high_dim*high_dim).reshape(high_dim,high_dim)  # <- これをなくしたい！
    E_[0:high_dim, 0:low_dim + 1] = Esub
    return lambda A_1, A_2: lam_f(q_, P_i, E_, A_1, A_2)

arr_init = np.array([1,0,0,1,0,0,0,0])  # どう一般化するか？
print("lambda: ready")

lambda: ready


In [8]:
######## Graph Drawing ########
root = Tk()
w = Canvas(root, width=_width, height=_height, bg='White')
w.pack()
circles = []
lines = []
r = 10

# 初期描画
for e in EdgeList:
	lines.append(w.create_line(Pos_scaled[0,e[0]-1], Pos_scaled[1,e[0]-1],Pos_scaled[0,e[1]-1], Pos_scaled[1,e[1]-1], fill='Black', tags='edge'))
for i in range(node_num):
	circles.append(w.create_oval(Pos_scaled[0,i] - r, Pos_scaled[1,i] - r, Pos_scaled[0,i] + r, Pos_scaled[1,i] + r, fill="White", tags='node'))

# 更新
def move_node(event):
    global Es
    x2 = unscale(event.x,True)
    y2 = unscale(event.y,False)
    if(low_dim == 3): return 0
    position = x2*eH + y2*eV
    thisID = event.widget.find_withtag(CURRENT)[0] - (edge_num+1)
    #E_0 = MDS[thisID] - (x2 * Es[:,0] + y2 * Es[:,1])
    E_0 = MDS[thisID] - np.sum(position.dot(Es[:, 0:low_dim].T), axis=1)
    Es[:,low_dim] = E_0 / np.linalg.norm(E_0)
    f2 = lam(position, MDS[thisID].reshape(1, high_dim), Es)
    def g(args):
        """
        最適化関数
        :param args: 行列を行ベクトルに崩したもの
        :return: f2 に args を叩き込んだもの
        """
        arr1 = args[0:low_dim*(low_dim+1)].reshape(low_dim+1, low_dim)  # E' variables ( E'[0] = this[0:dim+1,0] * (E:E0) )
        arr2 = args[low_dim*(low_dim+1):2*low_dim**2].reshape(low_dim, low_dim-1)  # R  variables ( ignore )
        return f2(arr1, arr2)
    res = opt.minimize(g, arr_init, method='L-BFGS-B')  # ,options={'ftol':1e-3}
    if(res.success):
        Coefficient = res.x[0:(low_dim+1)*low_dim].reshape(low_dim+1,low_dim)
        Es[:,0:low_dim] = Es.dot(Coefficient)
        update_points()
        for i in range(node_num):
            w.coords(circles[i], Pos_scaled[0,i] - r, Pos_scaled[1,i] - r, Pos_scaled[0,i] + r, Pos_scaled[1,i] + r)
        for i in range(edge_num):
            w.coords(lines[i], Pos_scaled[0,EdgeList[i][0] - 1], Pos_scaled[1,EdgeList[i][0] - 1], Pos_scaled[0,EdgeList[i][1] - 1], Pos_scaled[1,EdgeList[i][1] - 1])

# バインディング
w.tag_bind('node', '<Button1-Motion>', move_node)
root.mainloop()